# Greedy Technique

Given an array of size n that contain either ‘G’rab car or ‘P’assenger. 

Each Grab car can pick up only one passenger. And each Grab car cannot pick up a passenger who is more than K units away from Grab car.

1. Write a program using Brute force approach to find number of all solutions 
    that give the maximum number of Passenger(s) that can ride Grab(s).
2. Write a program using Greedy Technique to find a solution that gives the maximum number of Passengers(s) that can ride Grab(s).

For example, if an array consists of {‘G’, ‘P’, ‘P’, ‘G’, ‘P’} and we set k = 1, 

then the output the maximum number passenger can ride Grab would be 2. 

The first Grab picks up the first passenger and the second Grab picks up either the second or third passenger.

---

## Reading test case

In [28]:
def read_input(filename):
    with open(filename, 'r') as f:
        arr = list(f.readline().strip())
        k = int(f.readline().strip())
    return arr, k

# The functions 'can_pickup', 'recursive_search' and 'find_max_pickups' remain unchanged from the previous answer

# Reading the input
filename = 'Example_LAB_3.txt'
arr, k = read_input(filename)
print(f"arr = {arr}\nk   = {k}")

arr = ['P', 'P', 'G', 'G', 'G', 'G', 'P', 'P', 'P']
k   = 3


### Brute force

In [29]:
from itertools import combinations, product

def can_pickup(g_index, p_index, k):
    """Checks if a Grab car at g_index can pick up a passenger at p_index"""
    return abs(g_index - p_index) <= k

def brute_force(arr, k):
    g_indices = [i for i, x in enumerate(arr) if x == 'G']
    p_indices = [i for i, x in enumerate(arr) if x == 'P']
    max_pickups = 0
    best_solutions = []

    for r in range(1, min(len(g_indices), len(p_indices)) + 1):
        for pairs in combinations(product(g_indices, p_indices), r):

            grab_car_indices = [pair[0] for pair in pairs]
            passenger_indices = [pair[1] for pair in pairs]

            # Ensure all grab cars and passengers in the pairs are unique
            unique_grabs = len(set(grab_car_indices)) == r
            unique_passengers = len(set(passenger_indices)) == r

            if unique_grabs and unique_passengers:
                valid_pairs = all(can_pickup(grab, passenger, k) for grab, passenger in pairs)
                if valid_pairs:
                    if r > max_pickups:
                        max_pickups = r
                        best_solutions = [pairs]
                    elif r == max_pickups:
                        best_solutions.append(pairs)

    return max_pickups, best_solutions

max_passengers, solutions = brute_force(arr, k)

print(f"arr = {arr}\nk   = {k}")
print(f"Maximum passengers: {max_passengers}")
print(f"all possible solutions: {len(solutions)}")

arr = ['P', 'P', 'G', 'G', 'G', 'G', 'P', 'P', 'P']
k   = 3
Maximum passengers: 4
all possible solutions: 12


In [30]:
def calculate_distance(solution):
    """Calculate the total distance for a given solution."""
    return sum(abs(g - p) for g, p in solution)

def get_solutions_for_distance(solutions, distances, target_distance):
    """Get solutions corresponding to a specific distance."""
    return [solution for solution, distance in zip(solutions, distances) if distance == target_distance]

def print_solutions(title, solutions_list, distance=None):
    """Print a list of solutions with a title."""
    if distance is not None:
        print(f"{title} Distance: {distance}")
    for solution in solutions_list:
        print(solution)
    print("\n" + "-"*50 + "\n")

# Calculate distances for each solution
distances = [calculate_distance(solution) for solution in solutions]

# Find minimum and maximum distances
min_distance = min(distances)
max_distance = max(distances)

# Extract solutions with minimum and maximum distances
solutions_with_min_distance = get_solutions_for_distance(solutions, distances, min_distance)
solutions_with_max_distance = get_solutions_for_distance(solutions, distances, max_distance)

print(f"arr = {arr}\nk   = {k}")
# Print all solutions with their distances
for idx, (solution, distance) in enumerate(zip(solutions, distances), 1):
    print(f"Solution {idx}: {solution}, Total Distance: {distance}")

print("\n" + "-"*50 + "\n")

# Print solutions with minimum and maximum distances
print_solutions("Minimum", solutions_with_min_distance, min_distance)
print_solutions("Maximum", solutions_with_max_distance, max_distance)

arr = ['P', 'P', 'G', 'G', 'G', 'G', 'P', 'P', 'P']
k   = 3
Solution 1: ((2, 0), (3, 1), (4, 6), (5, 7)), Total Distance: 8
Solution 2: ((2, 0), (3, 1), (4, 6), (5, 8)), Total Distance: 9
Solution 3: ((2, 0), (3, 1), (4, 7), (5, 6)), Total Distance: 8
Solution 4: ((2, 0), (3, 1), (4, 7), (5, 8)), Total Distance: 10
Solution 5: ((2, 0), (3, 6), (4, 1), (5, 7)), Total Distance: 10
Solution 6: ((2, 0), (3, 6), (4, 1), (5, 8)), Total Distance: 11
Solution 7: ((2, 0), (3, 6), (4, 7), (5, 8)), Total Distance: 11
Solution 8: ((2, 1), (3, 0), (4, 6), (5, 7)), Total Distance: 8
Solution 9: ((2, 1), (3, 0), (4, 6), (5, 8)), Total Distance: 9
Solution 10: ((2, 1), (3, 0), (4, 7), (5, 6)), Total Distance: 8
Solution 11: ((2, 1), (3, 0), (4, 7), (5, 8)), Total Distance: 10
Solution 12: ((2, 1), (3, 6), (4, 7), (5, 8)), Total Distance: 10

--------------------------------------------------

Minimum Distance: 8
((2, 0), (3, 1), (4, 6), (5, 7))
((2, 0), (3, 1), (4, 7), (5, 6))
((2, 1), (3, 0), (4, 6),

---

### Greedy

In [31]:
def greedy_max_passenger(arr, k):
    n = len(arr)
    paired = [False] * n  # To mark passengers that have been paired
    pairs = []

    # Count number of G's within k distance for each P
    p_count = [0] * n
    for i in range(n):
        if arr[i] == 'P':
            for j in range(max(0, i-k), min(n, i+k+1)): # min max used to avoid out of bounds
                if arr[j] == 'G':
                    p_count[i] += 1

    #looking for Grab cars ('G') to pair them up.
    for i in range(n):
        if arr[i] == 'G':
            # Prioritize P with fewest G's around; if tie, pick nearest P
            min_count = float('inf') # used to track the minimum count of available Grab cars for any passenger
            chosen_p = None # will store the index of the chosen passenger
            for j in range(max(0, i-k), min(n, i+k+1)): # เอาไว้หาว่า G นี้มี P ที่น้อยที่สุดอยู่ที่ไหน
                if arr[j] == 'P' and not paired[j] and p_count[j] < min_count:
                    min_count = p_count[j]
                    chosen_p = j

            if chosen_p is not None:
                pairs.append((i, chosen_p))
                paired[chosen_p] = True

    return len(pairs), pairs

max_passengers, solution = greedy_max_passenger(arr, k)

print(f"arr = {arr}\nk   = {k}")
print(f"Maximum passengers: {max_passengers}")
print(f"Solution: {solution}")
distances = [calculate_distance(solution) for solution in solutions]
print(f"Total distance: {calculate_distance(solution)}")


arr = ['P', 'P', 'G', 'G', 'G', 'G', 'P', 'P', 'P']
k   = 3
Maximum passengers: 4
Solution: [(2, 0), (3, 1), (4, 7), (5, 8)]
Total distance: 10


----